# 08 - Resonant Collinearity

https://adventofcode.com/2024/day/8


In [6]:
// Read Input
const file = await Deno.readTextFile("input-base.txt");

const antennas = []
const area = file.split("\n").map((line, i) => line.split('').map((element, j) => {
    if (/[a-zA-Z0-9]/.test(element)) antennas.push([i, j])
    return element;
}));
[antennas, area]

[
  [
    [ 1, 8 ], [ 2, 5 ],
    [ 3, 7 ], [ 4, 4 ],
    [ 5, 6 ], [ 8, 8 ],
    [ 9, 9 ]
  ],
  [
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      "0", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", "0", ".", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", "0",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      "0", ".", ".", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", "A", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      ".", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      "A", ".", ".", "."
    ],
    [
      ".", ".", ".", ".",
      ".", ".", ".", ".",
      ".", "A", ".", "."
    ],


In [7]:
// Prepare Data

const maxI = area.length - 1;
const maxJ = area[0].length - 1;

const antennaGroups = antennas.reduce((acc, [i, j]) => {
    const key = area[i][j];
    if (!acc[key]) {
        acc[key] = [];
    }
    acc[key].push([i, j]);
    return acc;

}, {});

[maxI, maxJ, antennaGroups]


[
  11,
  11,
  {
    "0": [ [ 1, 8 ], [ 2, 5 ], [ 3, 7 ], [ 4, 4 ] ],
    A: [ [ 5, 6 ], [ 8, 8 ], [ 9, 9 ] ]
  }
]

In [8]:
// Part 1 - How many unique locations within the bounds of the map contain an antinode?

const antiNodes = new Array(area.length).fill(false).map(() => new Array(area[0].length).fill(false));
const findEquidistantExtendedPointOnLine = (pointA, pointB) => {
    const dx = pointB[0] - pointA[0];
    const dy = pointB[1] - pointA[1];
    return [pointB[0] + dx, pointB[1] + dy];
}

const findExtendedCollinearPoints = (pointA, pointB) => {
    const prePoint = findEquidistantExtendedPointOnLine(pointA, pointB);
    const postPoint = findEquidistantExtendedPointOnLine(pointB, pointA);
    return [prePoint, postPoint];
}

Object.keys(antennaGroups).forEach((group) => {
    const pairs = antennaGroups[group];
    if (pairs.length === 1) return;
    for (let i = 0; i < pairs.length; i += 1) {
        for (let j = i + 1; j < pairs.length; j += 1) {
            const extendedPoints = findExtendedCollinearPoints(pairs[i], pairs[j]);
            extendedPoints.forEach(point => {
                if (point[0] >= 0 && point[0] <= maxI && point[1] >= 0 && point[1] <= maxJ) {
                    antiNodes[point[0]][point[1]] = true;
                }
            });

        }
    }
});
console.table(antiNodes)
antiNodes.reduce((acc, row) => { return acc += row.filter(el => el).length }, 0);

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ (idx) │ 0     │ 1     │ 2     │ 3     │ 4     │ 5     │ 6     │ 7     │ 8     │ 9     │ 10    │ 11    │
├───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│     0 │ false │ false │ false │ false │ false │ false │ true  │ false │ false │ false │ false │ true  │
│     1 │ false │ false │ false │ true  │ false │ false │ false │ false │ false │ false │ false │ false │
│     2 │ false │ false │ false │ false │ true  │ false │ false │ false │ false │ false │ true  │ false │
│     3 │ false │ false │ true  │ false │ false │ false │ false │ false │ false │ false │ false │ false │
│     4 │ false │ false │ false │ false │ false │ false │ false │ false │ false │ true  │ false │ false │
│     5 │ false │ true  │ false │ false │ false │ false │ true  │ false │ false │ false │ false │ false │
│     6 │ false │ false │ false │ true  │ fals

14

In [9]:
// part 2 - How many unique locations within the bounds of the map contain an antinode resonant harmonics?

const antiNodesV2 = new Array(area.length).fill(false).map(() => new Array(area[0].length).fill(false));

const findAllExtendedCollinearPointsWithBounds = (pointA, pointB, maxI, maxJ) => {
    const points = [];

    const dx = pointB[0] - pointA[0];
    const dy = pointB[1] - pointA[1];

    for (let multiplier = 1; ; multiplier += 1) {
        const point = [pointB[0] + multiplier * dx, pointB[1] + multiplier * dy];
        if (point[0] < 0 || point[0] > maxI || point[1] < 0 || point[1] > maxJ) {
            break;
        }
        points.push(point);
    }
    return points;
}

Object.keys(antennaGroups).forEach((group) => {
    const pairs = antennaGroups[group];
    if (pairs.length === 1) return;
    for (let i = 0; i < pairs.length; i += 1) {
        for (let j = i + 1; j < pairs.length; j += 1) {
            const extendedPrePoints = findAllExtendedCollinearPointsWithBounds(pairs[i], pairs[j], maxI, maxJ);
            const extendedPostPoints = findAllExtendedCollinearPointsWithBounds(pairs[j], pairs[i], maxI, maxJ);
            [...extendedPrePoints, ...extendedPostPoints, pairs[i], pairs[j]].forEach(point => {
                if(!antiNodesV2[point[0]]) console.log(`point`, point)
                antiNodesV2[point[0]][point[1]] = true;
            });
        }
    }
});
console.table(antiNodesV2)
antiNodesV2.reduce((acc, row) => { return acc += row.filter(el => el).length }, 0);

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ (idx) │ 0     │ 1     │ 2     │ 3     │ 4     │ 5     │ 6     │ 7     │ 8     │ 9     │ 10    │ 11    │
├───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│     0 │ true  │ true  │ false │ false │ false │ false │ true  │ false │ false │ false │ false │ true  │
│     1 │ false │ true  │ false │ true  │ false │ false │ false │ false │ true  │ false │ false │ false │
│     2 │ false │ false │ true  │ false │ true  │ true  │ false │ false │ false │ false │ true  │ false │
│     3 │ false │ false │ true  │ true  │ false │ false │ false │ true  │ false │ false │ false │ false │
│     4 │ false │ false │ false │ false │ true  │ false │ false │ false │ false │ true  │ false │ false │
│     5 │ false │ true  │ false │ false │ false │ true  │ true  │ false │ false │ false │ false │ true  │
│     6 │ false │ false │ false │ true  │ fals

34

In [10]:
import colors from "../../utils/colors.ts";
colors.visualize2DArray(area, {})
colors.visualize2DArray(antiNodesV2, { colors : { false : colors.bgBlack, true: colors.bgRed }})